In [45]:
import pandas as pd
import math
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import networkx as nx
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import plot_tree
from sklearn.tree import DecisionTreeClassifier
import random

In [2]:
music_pd=pd.read_csv('full_music_data.csv')
music_pd

,artist_names,artists_id,danceability,energy,valence,tempo,loudness,mode,key,acousticness,instrumentalness,liveness,speechiness,explicit,duration_ms,popularity,year,release_date,song_title (censored)
0,"[""Fat Freddy's Drop""]",[178301],0.600,0.365,0.131,130.046,-13.083,0,9,0.06720,0.585000,0.0921,0.0498,0,437200,54,2005,2005,Ernie
1,"[""Fat Freddy's Drop""]",[178301],0.874,0.326,0.179,119.620,-13.302,0,11,0.01360,0.148000,0.0993,0.1310,0,581008,53,2005,2005,Wandering Eye
2,"[""Fat Freddy's Drop""]",[178301],0.670,0.531,0.336,139.385,-8.267,0,9,0.01560,0.345000,0.3060,0.0377,0,431293,55,2009,8/7/2009,The Raft
3,"[""Alexander O'Neal""]",[625201],0.761,0.702,0.850,104.773,-8.523,1,7,0.10800,0.000031,0.0935,0.0389,0,304427,34,1991,1/1/1991,All True Man
4,"[""Alexander O'Neal""]",[625201],0.661,0.828,0.902,115.078,-12.673,0,11,0.27600,0.000001,0.2870,0.0390,0,264933,37,1987,7/29/1987,(What Can I Say) To Make You Love Me
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98335,['ZZ Top'],[690254],0.276,0.892,0.715,80.475,-7.035,1,11,0.40900,0.000000,0.7140,0.0893,0,115973,33,1975,4/18/1975,Jailhouse Rock - **** Remaster
98336,['ZZ Top'],[690254],0.700,0.592,0.906,109.847,-10.434,1,11,0.10800,0.018400,0.1180,0.0681,0,263627,32,1976,11/29/1976,It's Only Love
98337,['ZZ Top'],[690254],0.709,0.709,0.863,111.544,-12.023,1,2,0.11800,0.000011,0.1250,0.0370,0,158400,31,1987,1987,Balinese
98338,['ZZ Top'],[690254],0.552,0.651,0.533,161.548,-10.624,0,4,0.00494,0.034900,0.1320,0.2290,0,232533,43,1992,4/13/1992,La Grange - **** Remaster


In [3]:
music_pd[['artists_id']]=music_pd[['artists_id']].apply(lambda x: x[0].lstrip("[").rstrip("]") if "," not in x[0] else np.nan,axis=1)
music_pd=music_pd.dropna(how='any').reset_index().drop(['index'],axis=1)

In [4]:
drop_list=['artist_names','popularity','year','release_date','song_title (censored)']
music_pd=music_pd.drop(drop_list,axis=1)
music_pd

,artists_id,danceability,energy,valence,tempo,loudness,mode,key,acousticness,instrumentalness,liveness,speechiness,explicit,duration_ms
0,178301,0.600,0.365,0.131,130.046,-13.083,0,9,0.06720,0.585000,0.0921,0.0498,0,437200
1,178301,0.874,0.326,0.179,119.620,-13.302,0,11,0.01360,0.148000,0.0993,0.1310,0,581008
2,178301,0.670,0.531,0.336,139.385,-8.267,0,9,0.01560,0.345000,0.3060,0.0377,0,431293
3,625201,0.761,0.702,0.850,104.773,-8.523,1,7,0.10800,0.000031,0.0935,0.0389,0,304427
4,625201,0.661,0.828,0.902,115.078,-12.673,0,11,0.27600,0.000001,0.2870,0.0390,0,264933
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93925,690254,0.276,0.892,0.715,80.475,-7.035,1,11,0.40900,0.000000,0.7140,0.0893,0,115973
93926,690254,0.700,0.592,0.906,109.847,-10.434,1,11,0.10800,0.018400,0.1180,0.0681,0,263627
93927,690254,0.709,0.709,0.863,111.544,-12.023,1,2,0.11800,0.000011,0.1250,0.0370,0,158400
93928,690254,0.552,0.651,0.533,161.548,-10.624,0,4,0.00494,0.034900,0.1320,0.2290,0,232533


In [5]:
influence_pd=pd.read_csv('influence_data.csv')
influence_pd1=influence_pd[['influencer_id','influencer_main_genre']]
influence_pd1.columns=['artists_id','genre']
influence_pd2=influence_pd[['follower_id','follower_main_genre']]
influence_pd2.columns=['artists_id','genre']
influence_pd1=pd.concat([influence_pd1,influence_pd2],axis=0)
influence_pd1=influence_pd1.groupby('artists_id').agg({
    'genre':'first'
})
influence_pd1=influence_pd1.reset_index()

In [6]:
influence_pd1[['artists_id']]=influence_pd1[['artists_id']].astype(np.int64)
music_pd[['artists_id']]=music_pd[['artists_id']].astype(np.int64)

In [7]:
music_pd=pd.merge(music_pd,influence_pd1,on='artists_id',how='left')
music_pd=music_pd.dropna(how='any')
music_pd=music_pd.reset_index().drop(['index','artists_id'],axis=1)

In [8]:
rd=np.random.permutation(music_pd.shape[0])
music_pd=music_pd.loc[rd]

In [9]:
music_pd=music_pd.reset_index().drop(['index'],axis=1)

In [10]:
music_pd
x=0.8
trX=music_pd[0:int(music_pd.shape[0]*x)]
teX=music_pd[trX.shape[0]:music_pd.shape[0]]

In [11]:
trY=trX[['genre']]
trX=trX.drop(['genre'],axis=1)
teY=teX[['genre']]
teX=teX.drop(['genre'],axis=1)

In [12]:
trX=np.array(trX)
teX=np.array(teX)
trY=np.resize(np.array(trY),(trY.shape[0],))
teY=np.resize(np.array(teY),(teY.shape[0],))

In [62]:
bdt = AdaBoostClassifier(DecisionTreeClassifier(max_depth=100,min_samples_split=10, min_samples_leaf=5),
                         algorithm="SAMME",
                         n_estimators=200, learning_rate=0.8)
bdt.fit(trX,trY)

AdaBoostClassifier(algorithm='SAMME',
                   base_estimator=DecisionTreeClassifier(max_depth=100,
                                                         min_samples_leaf=5,
                                                         min_samples_split=10),
                   learning_rate=0.8)

In [63]:
bdt.score(trX,trY)

1.0

In [64]:
bdt.score(teX,teY)
# 0.6528587780018532 100 200 0.8

0.6396141058483675

In [66]:
ar=bdt.feature_importances_
# array([0.08827968,        nan, 0.08711195,        nan, 0.09437133,
#        0.00880775, 0.04625925, 0.09894087, 0.07333423, 0.08984841,
#        0.09536723, 0.00119767, 0.12619632])

In [70]:
with open("importance.txt","w+",encoding='utf-8') as f:
    ans=""
    for col in music_pd.columns:
        ans+=col+","
    f.write(ans+"\n")
    ans=""
    for i in ar:
        ans+=str(i)+","
    f.write(ans+"\n")

,danceability,energy,valence,tempo,loudness,mode,key,acousticness,instrumentalness,liveness,speechiness,explicit,duration_ms,genre
0,0.496,0.0742,0.385,57.318,-14.670,1,1,0.993000,0.871000,0.1210,0.0512,0,195960,Jazz
1,0.503,0.6930,0.836,147.252,-12.043,1,2,0.125000,0.320000,0.0521,0.0398,0,253027,Pop/Rock
2,0.478,0.7280,0.741,96.656,-4.138,0,7,0.066400,0.000419,0.2900,0.0345,0,407533,Latin
3,0.541,0.6780,0.713,77.478,-7.624,1,2,0.556000,0.000000,0.0611,0.0319,0,181547,Pop/Rock
4,0.283,0.8400,0.293,145.275,-9.625,1,7,0.000401,0.001590,0.2050,0.0824,0,207933,Pop/Rock
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91726,0.484,0.9790,0.683,111.396,-6.986,0,4,0.000288,0.892000,0.0470,0.0404,0,186960,Pop/Rock
91727,0.125,0.7370,0.190,189.674,-7.349,1,0,0.000014,0.441000,0.0754,0.0868,0,216933,Pop/Rock
91728,0.654,0.4770,0.677,119.066,-10.388,1,2,0.737000,0.000020,0.4670,0.0298,0,197880,Country
91729,0.744,0.2510,0.709,115.668,-17.245,0,0,0.989000,0.523000,0.1220,0.0411,0,169760,Latin
